# Grounding in Logic Tensor Networks (LTN)

LTN consists of a non-logical part (the signature) and logical connectives and quantifiers. 
* **constants** denote individuals from some space of real numbers $\mathbb{R}^k$. We call this the **domain**
* **functions** can be any mathematical function either pre-defined or learnable. Examples of functions can be distance functions, similarities etc. Functions can be defined using any computational graph in Tensorflow. They can be linear functions, Deep Neural Networks.
* **predicates** are represented as functions that map from some n-ary domain to a real from $[0,1]$.
* **connectives** are modeled using t-norms
* **quantifiers** are based on domain sample aggregations

This tutorial explains how these concepts are grounded.

In [1]:
# initialization. we need numpy and matplotlib
# this tutorial uses the LTN wrapper
import logging;logging.basicConfig(level=logging.INFO);
import numpy as np;
import matplotlib.pyplot as plt; 
import tensorflow as tf;

# adding parent directory to sys.path (ltnw is in parent dir)
import sys, os
sys.path.insert(0,os.path.normpath(os.path.join(os.path.abspath(''),os.path.pardir)))
import logictensornetworks_wrapper as ltnw;
ltnw.SESSION=tf.InteractiveSession() # we will use an interactive session

### Constants 
Constants are grounded in some space of real numbers. Each constant $c$ is mapped to a point in $\mathbb{R}^k$. So that $c^\mathcal{G} \in\mathbb{R}^k$.
Here we define $c^\mathcal{G}=\left<2.1,3\right>$ and $d^\mathcal{G}=\left<3.4,1.5\right>$

In [2]:
ltnw.constant("c",[2.1,3.]);
ltnw.constant("d",[3.4,1.5]);

LTN Constants are in the simplest case represented by Tensorflow constants (or in some cases by Tensorflow variables). That means we can query the value of some constant easily.

In [3]:
ltnw.constant("c").eval()

array([2.1, 3. ], dtype=float32)

### Functions
LTN functions can be defined in any way. The following defines the grounding $\mathcal{G}$ of the function $f$ as $f^\mathcal{G}: \mathbb{R}^2\times \mathbb{R}^2\rightarrow \mathbb{R}^2$, with 
* $f^\mathcal{G}:\vec{x},\vec{y}\mapsto \vec{x} - \vec{y}$ 
* for $\vec{x},\vec{y}\in\mathbb{R}^2$.

In [4]:
ltnw.function("f",4,2,fun_definition=lambda x,y: x-y);

LTN functions are just computational graphs that can be called with constants, other functions and variables (explained later)

In [5]:
ltnw.term("f(c,d)").eval()

array([-1.3000002,  1.5      ], dtype=float32)

### Predicates

LTN Predicates are grounded in mappings that assign a value between zero and one to some n-ary space of input values. $P: \mathbb{R}^k\times \mathbb{R}^k ... \rightarrow \mathbb{R}$. Predicates in LTN can be Neural Tensor Networks or any other function that achieves such a mapping. The following defines a predicate $P$ using the similarity to point $\vec{\mu}=\left<2,3\right>$ and $P^\mathcal{G}:\vec{x}\mapsto \exp(-\|\vec{x}-\vec{\mu} \|^2)$

In [6]:
mu=tf.constant([2.,3.])
ltnw.predicate("P",2,pred_definition=lambda x: tf.exp(-tf.norm(tf.subtract(x,mu),axis=1)));

Predicates are just computational graphs, so we can easily query them. We use `ltnw.formula` for creating the formula (a Tensorflow graph) from a string. Then we call `eval` to let Tensorflow evaluate the graph and produce the satisfiability value.

In [7]:
ltnw.formula("P(c)").eval()

array([0.9048375], dtype=float32)

In [8]:
ltnw.formula("P(d)").eval()

array([0.12849975], dtype=float32)

In [9]:
ltnw.formula("P(f(c,d))").eval()

array([0.02665139], dtype=float32)

### Connectives

LTN suppports various logical connectives
* not: $\sim$ (logical symbol $\neg$)
* and: $\&$ (logical symbol $\wedge$)
* or: $|$ (logical symbol $\vee$)
* implication: $->$ (logical symbol $\rightarrow$)

These are grounded in Fuzzy Logic t-norms. 

In [10]:
ltnw.formula("~P(c)").eval()

array([0.09516251], dtype=float32)

In [11]:
ltnw.formula("P(c)&P(d)").eval()

array([0.03333712], dtype=float32)

In [12]:
ltnw.formula("P(c)|P(d)").eval()

array([1.], dtype=float32)

In [13]:
ltnw.formula("~P(f(c,d))->P(d)").eval()

array([0.15515113], dtype=float32)

### Quantifiers

LTN suppports two quantifiers. In order to ground these, let's first introduce variables.

LTN variables are sets of individuals/constants from a domain. By convention, we start variable labels with a question mark. The following defines two variables with 100 random samples each (from the domain). Notice that the set of values (i.e. individuals from the domain) for the two variables are different. 

In [14]:
ltnw.variable("?x",np.random.uniform(0.,4.,(100,2)).astype("float32"));
ltnw.variable("?y",np.random.uniform(0.,4.,(100,2)).astype("float32"));

Now we can evaluate the predicate $P$ for all $x$

In [15]:
ltnw.ask("P(?x)").shape

(100, 1)

We can then ask if for all points $x$, $y$, $P(f(x,y))$ is true. Notice that the outcome of this is a three dimensional tensor where each cell represents the satisfiability of $P(f(a,b))$ with $a\in x$ and $b\in y$

In [16]:
ltnw.ask("P(f(?x,?y))").shape

(100, 100, 1)

In [17]:
ltnw.ask("P(f(?x,?y)) | ~P(?x)").shape

(100, 100, 1)

Quantifiers can be used to make statements about such three multidimensional tensors by essentially margining out variables. For instance, the following checks if $P(x)$ is true for all $x$

In [18]:
ltnw.ask("forall ?x: P(?x)")

array([0.11074361], dtype=float32)

Or if there exists a sample for which $P$ holds

In [19]:
ltnw.ask("exists ?x: P(?x)")

array([0.8691504], dtype=float32)